# Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pyomo.environ as pyo
from __future__ import division

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Route Optimization\\Pyomo'

# Pyomo Overview

## Abstract vs. Concrete Models

A mathematical model can be defined using symbols that represent data values. For example, the following equations represent a linear program (LP) to find optimal values for the vector $x$ with parameters $n$ and $b$, and parameter vectors $a$ and $c$:

$$ \begin{align}
\text{min}   & \sum^{n}_{j = 1} c_jx_j               & \\

\text{s.t.}         & \sum^{n}_{j = 1} a_{ij}x_j \geq b_i   & \forall i = 1 \dots m \\
             & x_j \geq 0                            & \forall j = 1 \dots n
\end{align}
$$

We call this an *abstract* or *symbolic* mathematical model since it relies on unspecified parameter values. Data values can be used to specify a *model instance*. The `AbstractModel` class provides a context for defining and initializing abstract optimization models in Pyomo when the data values will be supplied at the time a solution is to be obtained. 

In many contexts, a mathematical model can and should be directly defined with the data values supplied at the time of the model definition. We call these *concrete* mathematical models. For example, the following LP model is a concrete instance of the previous abstract model:

$$ \begin{align}
\text{min }          & 2x_1 + 3x_2 \\
\text{s.t. }         & 3x_1 + 4x_2 \geq 1 \\
                    & x_1, x2 \geq 0
\end{align}
$$

The `ConcreteModel` class is used to define concrete optimization models in Pyomo.

## Simple Models

We can showcase the concrete model here.

In [3]:
model = pyo.ConcreteModel()
model.x = pyo.Var([1, 2], domain = pyo.NonNegativeReals)
model.OBJ = pyo.Objective(expr = 2 * model.x[1] + 3 * model.x[2])
model.Constraint1 = pyo.Constraint(expr = 3 * model.x[1] + 4 * model.x[2] >= 1)

We can also showcase the abstract model here.

In [6]:
model = pyo.AbstractModel()

model.m = pyo.Param(within = pyo.NonNegativeIntegers)
model.n = pyo.Param(within = pyo.NonNegativeIntegers)

model.I = pyo.RangeSet(1, model.m)
model.J = pyo.RangeSet(1, model.n)

model.a = pyo.Param(model.I, model.J)
model.b = pyo.Param(model.I)
model.c = pyo.Param(model.J)

# Declare a variable indexed by the set J
model.x = pyo.Var(model.J, domain = pyo.NonNegativeReals)

def obj_expression(m):
    return pyo.summation(m.c, m.x)

model.OBJ = pyo.Objective(rule = obj_expression)

def ax_constraint_rule(m, i):
    # Return expression for constraint for i
    return sum(m.a[i, j] * m.x[j] for j in m.J) >= m.b[i]

# Create one constraint for each member of the set
model.AxbConstraint = pyo.Constraint(model.I, rule = ax_constraint_rule)

# Pyomo Modeling Components

## Sets

Sets can be declared using instances of the `Set` and `RangeSet` classes or by assigning set expressions. The simplest set declaration creates a set and postpones creation of its members:

In [7]:
model.A = pyo.Set()

The `Set` class takes optional arguments such as:
- `dimen` = Dimension of the members of the set
- `doc` = String describing the set
- `filter` = A Boolean function used during construction to indicate if a potential new member should be assigned to the set
- `initialize` = An iterable containing the initial members of the Set, or function that returns an iterable of the initial members the set.
- `ordered` = A Boolean indicator that the set is ordered; the default is `True`
- `validate` = A Boolean function that validates new member data
- `within` = Set used for validation; it is a super-set of the set being declared.

In general, Pyomo attempts to infer the “dimensionality” of Set components (that is, the number of apparent indices) when they are constructed. However, there are situations where Pyomo either cannot detect a dimensionality (e.g., a `Set` that was not initialized with any members), or you the user may want to assert the dimensionality of the set. This can be accomplished through the dim`en keyword. For example, to create a set whose members will be tuples with two items, one could write:

In [8]:
model = pyo.Set(dimen = 2)

To create a set of all the numbers in set `model.A` doubles, one could use:

In [9]:
def DoubleA_init(model):
    return (i * 2 for i in model.A)

model.C = pyo.Set(initialize = DoubleA_init)

As an aside we note that as always in Python, there are lot of ways to accomplish the same thing. Also, note that this will generate an error if `model.A` contains elements for which multiplication times two is not defined.

The `initialize` option can accept any Python iterable, including a `set`, `list`, or `tuple`. This data may be returned from a function or specified directly as in

In [10]:
model.D = pyo.Set(initialize = ['red', 'green', 'blue'])

The `initialize` option can also specify either a generator or a function to specify the Set members. In the case of a generator, all data yielded by the generator will become the initial set members:

In [16]:
def X_init(m):
    for i in range(10):
        yield 2 * i + 1

model.X = pyo.Set(initialize = X_init)

def Y_init(m):
    return [2 * i + 1 for i in range(10)]
model.Y = pyo.Set(initialize = Y_init)

In the second signature, the function is called for each element, passing the element number in as an extra argument. This is repeated until the function returns the special value `Set.End`:

In [17]:
def Z_init(model, i):
    if i < 10:
        return pyo.Set.End
    return 2 * i + 1

model.Z = pyo.Set(initialize = Z_init)

Note that the element number starts with 1 and not 0.

In [18]:
model.X.pprint()

AbstractOrderedScalarSet : Size=0, Index=None, Ordered=Insertion
    Key : Dimen : Domain : Size : Members


In [19]:
model.Y.pprint()

AbstractOrderedScalarSet : Size=0, Index=None, Ordered=Insertion
    Key : Dimen : Domain : Size : Members


In [20]:
model.Z.pprint()

AbstractOrderedScalarSet : Size=0, Index=None, Ordered=Insertion
    Key : Dimen : Domain : Size : Members
